## Efficient IMDb Data Import and Filtering

We read the large IMDb `title.basics.tsv` file in chunks to avoid memory issues.  
Only entries where `titleType` is `'movie'` and `startYear` is present are kept.  
The filtered chunks are then combined into a single DataFrame.

In [2]:
import pandas as pd


usecols = ['tconst', 'titleType', 'primaryTitle', 'startYear', 'genres']


chunks = pd.read_csv(
    "../data/raw/title.basics-2.tsv",
    sep='\t',
    dtype=str,
    na_values='\\N',
    usecols=usecols,
    chunksize=100_000
)

filtered_chunks = []
for chunk in chunks:
    movies = chunk[chunk['titleType'] == 'movie']
    movies = movies[movies['startYear'].notna()]
    filtered_chunks.append(movies)


basics = pd.concat(filtered_chunks, ignore_index=True)


## Load and Prepare IMDb Ratings Data

We load the `title.ratings.tsv` file and convert the relevant columns to numeric types.  
This step prepares the `averageRating` and `numVotes` fields for further analysis and merging with movie data.

In [3]:
ratings = pd.read_csv(
    "../data/raw/title.ratings.tsv",
    sep='\t',
    dtype=str,
    na_values='\\N'
)

ratings = ratings[['tconst', 'averageRating', 'numVotes']]
ratings['averageRating'] = pd.to_numeric(ratings['averageRating'], errors='coerce')
ratings['numVotes'] = pd.to_numeric(ratings['numVotes'], errors='coerce')

## Merge Movie Information with Ratings

We join the cleaned movie data (`basics`) with the ratings data on the `tconst` identifier.  
This results in a combined dataset that includes title, year, genres, average rating, and number of votes for each film.


In [4]:
import sqlite3

conn = sqlite3.connect("../data/raw/imdb.sqlite")  
basics.to_sql("basics", conn, if_exists="replace", index=False)
ratings.to_sql("ratings", conn, if_exists="replace", index=False)
conn.close()